### 1. process_data(corpus_file)
Cette fonction lit un fichier texte, le convertit en minuscules, segmente le texte et retourne une liste de mots

In [8]:
import re

def process_data(corpus_file):
    with open(corpus_file, 'r', encoding='utf-8') as file:
        text = file.read().lower()
    words = re.findall(r'\b[a-z]+\b', text)
    return words

# Utilisation de la fonction avec le fichier fourni
corpus_file = 'big.txt'
words = process_data(corpus_file)
print(words[:10])  # Affiche les 10 premiers mots pour vérification


['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']


### 2. get_vocabulary(corpus_file)
Cette fonction retourne le vocabulaire construit à partir du corpus.

In [9]:
def get_vocabulary(corpus_file):
    words = process_data(corpus_file)
    return set(words)

# Utilisation de la fonction avec le fichier fourni
vocabulary = get_vocabulary(corpus_file)
print(list(vocabulary)[:10])  # Affiche 10 mots du vocabulaire pour vérification


['empresses', 'stump', 'smokers', 'matreshka', 'tightening', 'terror', 'seventy', 'wedlock', 'rejoice', 'parapet']


### 3. Modèle de langue
Nous allons estimer la probabilité de chaque mot en utilisant la fréquence des mots dans le corpus.

In [10]:
from collections import Counter

def build_language_model(corpus_file):
    words = process_data(corpus_file)
    total_words = len(words)
    word_counts = Counter(words)
    language_model = {word: count / total_words for word, count in word_counts.items()}
    return language_model

# Utilisation de la fonction avec le fichier fourni
language_model = build_language_model(corpus_file)
print(list(language_model.items())[:10])  # Affiche 10 mots avec leurs probabilités pour vérification


[('the', 0.07250018395528388), ('project', 0.0002616252926251645), ('gutenberg', 0.00023891476375145234), ('ebook', 7.903264048051846e-05), ('of', 0.036358648305658284), ('adventures', 1.5443159634124297e-05), ('sherlock', 9.175053664979728e-05), ('holmes', 0.0004242326793609439), ('by', 0.006118216478578066), ('sir', 0.00016079054442588237)]


### 4. Fonctions edits1, edits2 et knownWord
Ces fonctions génèrent les modifications possibles et filtrent les mots valides.

In [11]:
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def knownWord(words, vocabulary):
    return set(word for word in words if word in vocabulary)

# Vérification des fonctions
word = "example"
print(" l’ensemble de toutes les chaînes obtenues avec une seule modification (insertion, substitution ou suppression) \n")
print(edits1(word))
print("\n")
print("l’ensemble de toutes les chaînes obtenues avec une 2 modification (insertion, substitution ou suppression)\n ")
print(edits2(word))
print(knownWord(["example", "test", "words"], vocabulary))


 l’ensemble de toutes les chaînes obtenues avec une seule modification (insertion, substitution ou suppression) 

{'ekample', 'examjle', 'exampfe', 'examrle', 'examxle', 'excample', 'exampler', 'rxample', 'exyample', 'nexample', 'exarple', 'examole', 'exaiple', 'examgple', 'exmmple', 'exapmle', 'examuple', 'eoample', 'exzample', 'exrample', 'exacmple', 'epample', 'examdple', 'examplex', 'iexample', 'exampxe', 'examlple', 'examples', 'ezample', 'dxample', 'ekxample', 'examplek', 'excmple', 'examplq', 'examplx', 'exmaple', 'mxample', 'exaomple', 'exaople', 'examplxe', 'kexample', 'exagmple', 'examiple', 'examplj', 'exalple', 'pexample', 'fxample', 'esample', 'exampley', 'exaqple', 'lxample', 'exampln', 'ewxample', 'hxample', 'examplne', 'exampnle', 'ebample', 'exkample', 'exampee', 'exawmple', 'nxample', 'fexample', 'exampvle', 'eaxmple', 'exkmple', 'exakple', 'exampli', 'cxample', 'eixample', 'examplej', 'exdmple', 'exapmple', 'examplo', 'exampla', 'exampdle', 'examzle', 'exampleb', 'el

### 5. Fonction candidates
Cette fonction retourne les candidats les plus probables pour une correction.

In [13]:
def candidates(word, vocabulary):
    if word in vocabulary:
        return [word]
    candidates1 = knownWord(edits1(word), vocabulary)
    if candidates1:
        return list(candidates1)
    candidates2 = knownWord(edits2(word), vocabulary)
    if candidates2:
        return list(candidates2)
    return [word]

# Vérification de la fonction
print(candidates("exampel", vocabulary))


['example']


### 6. Fonction correction
Cette fonction retourne les k corrections les plus probables pour un mot donné.

In [14]:
def correction(word, k, language_model, vocabulary):
    possible_candidates = candidates(word, vocabulary)
    sorted_candidates = sorted(possible_candidates, key=lambda w: language_model.get(w, 0), reverse=True)
    return sorted_candidates[:k]

# Vérification de la fonction
print(correction("exampel", 3, language_model, vocabulary))


['example']


In [15]:
def correction(word, k, language_model, vocabulary):
    possible_candidates = candidates(word, vocabulary)
    sorted_candidates = sorted(possible_candidates, key=lambda w: language_model.get(w, 0), reverse=True)
    return sorted_candidates[:k]
print(correction("mial", 3, language_model, vocabulary))


['mill', 'meal', 'mail']


In [16]:

print(correction("porjet", 3, language_model, vocabulary))


['project', 'forget', 'pocket']


In [17]:
print(correction("porject", 3, language_model, vocabulary))


['project']


In [18]:
print(correction("proect", 3, language_model, vocabulary))


['project', 'protect']
